In [333]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm_notebook as tqdm

from sklearn.metrics import ndcg_score

import warnings

# Suppress future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# history_val_path = f'./files/parquet/ebnerd_{size}/validation/history.parquet'
# history_val_df = pd.read_parquet(history_val_path)
# print(f'Raw {size} validation history df shape:            ',history_val_df.shape)

# #Raw data
# size = 'large'
# articles_path = f'./files/parquet/ebnerd_{size}/articles.parquet'
# articles_df = pd.read_parquet(articles_path)
# print('Raw Articles df shape:                           ',articles_df.shape)

# Read history and behavior files
size = 'demo'
type_ = 'train'


behaviors_path = f'./files/parquet/ebnerd_{size}/{type_}/behaviors.parquet'
behaviors_df = pd.read_parquet(behaviors_path)
print(f'Raw {size} validation behaviors df shape:          ',behaviors_df.shape)

# Read cosine similarity matrix pickle file
# num_of_rows = 50000
# mult_way = 'full'
# cosine_similarity_df_file_path = f'./files/pickle/cosine_similarity_tail_{str(num_of_rows)}_{mult_way}.pkl'
cosine_similarity_df_file_path = f'./files/pickle/cosine_similarity.pkl'
cosine_similarity_df = pd.read_pickle(cosine_similarity_df_file_path)
print('Cosine similarity df shape:                      ',cosine_similarity_df.shape)

# # Read interaction matrix pickle file
# size = 'small'
# interaction_matrix_file_path = f'./files/pickle/interaction_matrix_{size}.pkl'
# interaction_matrix_df = pd.read_pickle(interaction_matrix_file_path)

# interaction_matrix_df = interaction_matrix_df.replace({np.nan: -999})
# print('Interaction df shape:                            ',interaction_matrix_df.shape)

# # Read user matrix and article matrix pickle file
# factorized_size = 'small'
# fillnan_value = 'mean_column'
# user_matrix_df_file_path = f'./files/pickle/user_matrix_{factorized_size}_fillnan_{fillnan_value}.pkl'
# article_matrix_df_file_path = f'./files/pickle/article_matrix_{factorized_size}_fillnan_{fillnan_value}.pkl'

# user_matrix_df = pd.read_pickle(user_matrix_df_file_path)
# article_matrix_df = pd.read_pickle(article_matrix_df_file_path)
# print('User embedding df shape:                         ',user_matrix_df.shape)
# print('Article embedding df shape:                      ',article_matrix_df.shape)



Raw demo validation behaviors df shape:           (24724, 17)
Cosine similarity df shape:                       (5638, 5638)


In [14]:
# print(history_val_df.explode('impression_time_fixed').impression_time_fixed.min())
# print(history_val_df.explode('impression_time_fixed').impression_time_fixed.max())
# history_val_df.head(2)

2023-05-04 07:00:11
2023-05-25 06:59:54


,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,750497,"[2023-05-04T09:42:39.000000, 2023-05-04T09:43:...","[100.0, 65.0, 100.0, 100.0, 100.0, 100.0, 100....","[9749224, 9749156, 9749224, 9748948, 9748980, ...","[49.0, 5.0, 7.0, 151.0, 214.0, 199.0, 22.0, 64..."
1,22779,"[2023-05-04T07:53:42.000000, 2023-05-04T15:59:...","[52.0, 39.0, 62.0, 38.0, 74.0, 19.0, 30.0, 56....","[9749025, 9750090, 9750015, 9750161, 9745750, ...","[4.0, 16.0, 2.0, 9.0, 40.0, 7.0, 9.0, 8.0, 18...."


In [337]:
print('Min impression timestamp: ', behaviors_df.impression_time.min())
print('Min impression timestamp: ', behaviors_df.impression_time.max())
behaviors_df.head(20)

Min impression timestamp:  2023-05-18 07:00:03
Min impression timestamp:  2023-05-25 06:59:52


,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
0,48401,NaN,2023-05-21 21:06:50,21.0,NaN,2,"[9774516, 9771051, 9770028, 9775402, 9774461, ...",[9759966],22779,False,NaN,NaN,NaN,False,21,16.0,27.0
1,152513,9778745.0,2023-05-24 07:31:26,30.0,100.0,1,"[9778669, 9778736, 9778623, 9089120, 9778661, ...",[9778661],150224,False,NaN,NaN,NaN,False,298,2.0,48.0
2,155390,NaN,2023-05-24 07:30:33,45.0,NaN,1,"[9778369, 9777856, 9778500, 9778021, 9778627, ...",[9777856],160892,False,NaN,NaN,NaN,False,401,215.0,100.0
3,214679,NaN,2023-05-23 05:25:40,33.0,NaN,2,"[9776715, 9776406, 9776566, 9776071, 9776808, ...",[9776566],1001055,False,NaN,NaN,NaN,False,1357,40.0,47.0
4,214681,NaN,2023-05-23 05:31:54,21.0,NaN,2,"[9775202, 9776855, 9776688, 9771995, 9776583, ...",[9776553],1001055,False,NaN,NaN,NaN,False,1358,5.0,49.0
5,214684,NaN,2023-05-23 05:32:21,10.0,NaN,2,"[9776508, 9767490, 9776049, 9776544, 9776551, ...",[9776508],1001055,False,NaN,NaN,NaN,False,1358,52.0,100.0
6,214691,NaN,2023-05-23 05:30:46,18.0,NaN,2,"[9759955, 9776449, 9775804, 9776369, 9488213, ...",[9776691],1001055,False,NaN,NaN,NaN,False,1358,4.0,37.0
7,369958,NaN,2023-05-24 14:25:56,16.0,NaN,2,"[9776023, 9778158, 9776929, 9527795, 7594265]",[9778158],1469458,False,NaN,NaN,NaN,False,1623,0.0,NaN
8,369959,NaN,2023-05-24 14:23:14,161.0,NaN,2,"[9779186, 9779289, 9777397, 9779185, 9778813, ...",[9779071],1469458,False,NaN,NaN,NaN,False,1623,16.0,NaN
9,370414,NaN,2023-05-24 14:48:54,9.0,NaN,2,"[9779408, 9779377, 9779289, 9778945, 9777182, ...",[9777182],1470585,False,NaN,NaN,NaN,False,1678,13.0,41.0


In [343]:
behaviors_df[behaviors_df['user_id']==1598988]

,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
12,386918,NaN,2023-05-20 20:41:22,26.0,NaN,2,"[9773465, 9774074, 9774120, 9774079, 9774096]",[9773465],1598988,False,NaN,NaN,NaN,False,2512,128.0,100.0
17549,415011452,9778915.0,2023-05-24 12:05:11,56.0,100.0,2,"[9779019, 9768150, 9767675, 9768764, 9775076, ...",[9778645],1598988,False,NaN,NaN,NaN,False,2516,5.0,12.0


In [346]:
article_id = 9778645

article_id in list(behaviors_df.loc[17549,:]['article_ids_inview'])

True

In [295]:
print(cosine_similarity_df.index.max())
cosine_similarity_df.head(2)

9780271


article_id,9306121,9568274,9764895,5996581,9666599,9764909,9764916,9732155,9699394,9666638,...,9732011,9764781,5341102,9764787,9764796,9764813,9732045,9764822,8617974,9764857
article_id,,,,,,,,,,,,,,,,,,,,,
9306121,1.000000,0.674356,0.747147,0.858023,0.876015,0.705622,0.744126,0.932712,0.840868,0.821475,...,0.913827,0.763160,0.930627,0.838673,0.753952,0.878267,0.813538,0.804546,0.760747,0.862884
9568274,0.674356,1.000000,0.631668,0.671629,0.609433,0.610673,0.595214,0.675617,0.600703,0.650559,...,0.720504,0.654405,0.639809,0.603827,0.622352,0.616112,0.673254,0.655378,0.679650,0.692131


In [37]:
# user_matrix_df.head(2)

In [307]:
# ----- NOT SURE IF THESE TRANSFORMATION ARE NEEDED NEITHER IF WE MAKE THE RECOMMENDATIONS IN THIS DATASET --------
behaviors_df = behaviors_df[['user_id','article_ids_inview', 'article_ids_clicked']]
behaviors_df = behaviors_df.explode('article_ids_clicked')

behaviors_val_grouped_clicked = df = behaviors_df.groupby('user_id')['article_ids_clicked'].apply(list).reset_index()

behaviors_df = behaviors_df.explode('article_ids_inview')

behaviors_val_grouped_inview_df = behaviors_df.groupby('user_id')['article_ids_inview'].apply(list).reset_index()

behaviors_df = pd.merge(behaviors_val_grouped_inview_df, behaviors_val_grouped_clicked, on='user_id', how='inner')

behaviors_df['article_ids_inview_setted_lst'] = behaviors_df['article_ids_inview'].apply(lambda lst: list(set(lst)))

behaviors_df['article_ids_inview_len'] = behaviors_df['article_ids_inview'].apply(lambda lst: len(lst))
behaviors_df['article_ids_inview_set_len'] = behaviors_df['article_ids_inview'].apply(lambda lst: len(list(set(lst))))
behaviors_df['article_ids_clicked_len'] = behaviors_df['article_ids_clicked'].apply(lambda lst: len(lst))

print(behaviors_df.shape)
behaviors_df.head(2)

(1590, 7)


,user_id,article_ids_inview,article_ids_clicked,article_ids_inview_setted_lst,article_ids_inview_len,article_ids_inview_set_len,article_ids_clicked_len
0,11313,"[9776968, 9776917, 9777036, 9776897, 9777005, ...","[9776897, 9779860, 9778627, 9776855, 9776715, ...","[9776897, 9779713, 9775489, 9779205, 9777036, ...",74,58,8
1,13538,"[9754160, 9775485, 9768866, 9142581, 9770028, ...","[9754160, 9771187, 9773768, 9772343, 9775171, ...","[9773070, 9773078, 9775142, 9770028, 9754160, ...",87,81,12


In [12]:
# Group by 'user_id' and aggregate 'article_ids_clicked' into lists
# behaviors_df = behaviors_df.groupby('user_id')['article_ids_inview'].apply(list).reset_index()
# behaviors_df.head()


In [171]:
cosine_similarity_df.head(5)

article_id,9568274,9764895,5996581,9666599,9764909,9764916,9732155,9699394,9666638,9470037,...,9732011,9764781,5341102,9764787,9764796,9764813,9732045,9764822,8617974,9764857
article_id,,,,,,,,,,,,,,,,,,,,,
9568274,1.000000,0.631668,0.671629,0.609433,0.610673,0.595214,0.675617,0.600703,0.650559,0.706415,...,0.720504,0.654405,0.639809,0.603827,0.622352,0.616112,0.673254,0.655378,0.679650,0.692131
9764895,0.631668,1.000000,0.724137,0.729785,0.809719,0.811588,0.739873,0.698493,0.676368,0.631210,...,0.769097,0.932155,0.705713,0.752221,0.941420,0.758056,0.915913,0.659229,0.607496,0.878707
5996581,0.671629,0.724137,1.000000,0.820750,0.657701,0.712269,0.877603,0.813125,0.860892,0.722785,...,0.850992,0.741205,0.892585,0.831485,0.717500,0.814572,0.791174,0.785255,0.688688,0.814857
9666599,0.609433,0.729785,0.820750,1.000000,0.681974,0.715353,0.893717,0.905076,0.763104,0.677125,...,0.843584,0.736213,0.898663,0.888842,0.728161,0.925385,0.796972,0.801073,0.700038,0.828615
9764909,0.610673,0.809719,0.657701,0.681974,1.000000,0.724609,0.692557,0.653429,0.651196,0.575875,...,0.728498,0.861528,0.671766,0.664602,0.860245,0.700445,0.784086,0.656889,0.668975,0.863056


In [297]:
article_id = 9776897

In [298]:
cosine_similarity_df

article_id,9306121,9568274,9764895,5996581,9666599,9764909,9764916,9732155,9699394,9666638,...,9732011,9764781,5341102,9764787,9764796,9764813,9732045,9764822,8617974,9764857
article_id,,,,,,,,,,,,,,,,,,,,,
9306121,1.000000,0.674356,0.747147,0.858023,0.876015,0.705622,0.744126,0.932712,0.840868,0.821475,...,0.913827,0.763160,0.930627,0.838673,0.753952,0.878267,0.813538,0.804546,0.760747,0.862884
9568274,0.674356,1.000000,0.631668,0.671629,0.609433,0.610673,0.595214,0.675617,0.600703,0.650559,...,0.720504,0.654405,0.639809,0.603827,0.622352,0.616112,0.673254,0.655378,0.679650,0.692131
9764895,0.747147,0.631668,1.000000,0.724137,0.729785,0.809719,0.811588,0.739873,0.698493,0.676368,...,0.769097,0.932155,0.705713,0.752221,0.941420,0.758056,0.915913,0.659229,0.607496,0.878707
5996581,0.858023,0.671629,0.724137,1.000000,0.820750,0.657701,0.712269,0.877603,0.813125,0.860892,...,0.850992,0.741205,0.892585,0.831485,0.717500,0.814572,0.791174,0.785255,0.688688,0.814857
9666599,0.876015,0.609433,0.729785,0.820750,1.000000,0.681974,0.715353,0.893717,0.905076,0.763104,...,0.843584,0.736213,0.898663,0.888842,0.728161,0.925385,0.796972,0.801073,0.700038,0.828615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9764813,0.878267,0.616112,0.758056,0.814572,0.925385,0.700445,0.743241,0.877010,0.891023,0.773386,...,0.863720,0.747371,0.863423,0.896758,0.742519,1.000000,0.836640,0.768665,0.694652,0.842205
9732045,0.813538,0.673254,0.915913,0.791174,0.796972,0.784086,0.816504,0.815755,0.778163,0.750562,...,0.839040,0.889164,0.794697,0.817002,0.873311,0.836640,1.000000,0.727525,0.678485,0.909706
9764822,0.804546,0.655378,0.659229,0.785255,0.801073,0.656889,0.682429,0.811877,0.801739,0.835375,...,0.809867,0.702192,0.821513,0.756528,0.677056,0.768665,0.727525,1.000000,0.738327,0.774176


In [299]:
cosine_similarity_df.loc[article_id, :].sort_values(ascending=False)

article_id
9776897    1.000000
9776041    0.894936
9776862    0.888198
9777007    0.883037
9777319    0.876222
             ...   
9709346    0.369688
9559884    0.362744
9386169    0.305716
9764062    0.290846
9734027    0.268555
Name: 9776897, Length: 5638, dtype: float32

In [246]:
df1 = cosine_similarity_df.loc[article_id, :].sort_values(ascending=False)[1:11].reset_index()

In [247]:
df1.rename({article_id:'cosine_similarity'}, axis=1, inplace=True)

In [248]:
df1

,article_id,cosine_similarity
0,9537767,0.963050
1,9380594,0.962837
2,9747633,0.962813
3,9644724,0.962246
4,9663715,0.960324
5,9754241,0.959723
6,9597707,0.958102
7,9487873,0.957351
8,9738844,0.956354
9,9759300,0.954273


In [281]:
df2 = cosine_similarity_df.loc[article_id, :].sort_values(ascending=False)[1:11].reset_index()
df2.rename({article_id:'cosine_similarity'}, axis=1, inplace=True)

In [282]:
df3 = pd.DataFrame(columns=['article_id', 'cosine_similarity'])

In [283]:
df3

,article_id,cosine_similarity


In [321]:
# Concatenate DataFrames vertically
df_final = pd.concat([df3, df2], ignore_index=True)


In [322]:
df_final

,article_id,cosine_similarity
0,9537767,0.963050
1,9380594,0.962837
2,9747633,0.962813
3,9644724,0.962246
4,9663715,0.960324
5,9754241,0.959723
6,9597707,0.958102
7,9487873,0.957351
8,9738844,0.956354
9,9759300,0.954273


In [325]:
df_final.sort_values(by='article_id', ascending=False, inplace=True).head(50)

In [326]:
df_final

,article_id,cosine_similarity
9,9759300,0.954273
5,9754241,0.959723
2,9747633,0.962813
8,9738844,0.956354
4,9663715,0.960324
3,9644724,0.962246
6,9597707,0.958102
0,9537767,0.963050
7,9487873,0.957351
1,9380594,0.962837


In [265]:
df_final.drop_duplicates(subset='article_id', keep='first')['article_id'].to_list()

[9380594,
 9487873,
 9537767,
 9597707,
 9644724,
 9663715,
 9738844,
 9747633,
 9754241,
 9759300]

In [320]:
# Function to get recommendations based on content similarity
# def get_recommendations(article_id, cosine_similarity_df=cosine_similarity_df):
#     #idx = articles_df[articles_df['article_id'] == article_id].index[0]
#     sim_scores = list(enumerate(cosine_similarity_df.loc[article_id, :]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:11]  # Top 10 recommendations
#     article_indices = [i[0] for i in sim_scores]
#     return articles_df['article_id'].iloc[article_indices]


# Example Ids
article_id = 9776897
user_id = 19181

def get_content_based_recommendations(article_id, cosine_similarity_df=cosine_similarity_df):
    try:
        # article_indices = cosine_similarity_df.loc[article_id, :].sort_values(ascending=False)[1:11].index.tolist()
        
        # category_str = cosine_similarity_df[cosine_similarity_df.index == article_id]['category_str'].values[0]

        # article_id_category_lst = cosine_similarity_df[cosine_similarity_df['category_str'] == category_str].index

        # article_indices = cosine_similarity_df[cosine_similarity_df.index == article_id].drop('category_str', axis=1).T.loc[article_id_category_lst, :].sort_values(article_id, ascending=False)[1:11].index.tolist()

        temp_df = cosine_similarity_df.loc[article_id, :].sort_values(ascending=False)[1:11].reset_index()
        temp_df.rename({article_id:'cosine_similarity'}, axis=1, inplace=True)
        return temp_df
    except:
        return [] # Work to be Done

print('Content based recomendations:                               ', get_content_based_recommendations(article_id))

# def get_collaborative_filtering_recommendations_new(user_id, user_matrix_df, article_matrix_df):
#     try:
#         user_pref = user_matrix_df.loc[user_id, :]
#         cf_interaction_scores = user_pref.dot(article_matrix_df.values.T)
#         cf_recs = pd.DataFrame(cf_interaction_scores, index=article_matrix_df.index).sort_values(0, ascending=False).index

#         return cf_recs
#     except:
#         print("The user is not present in the user pref")
#         return []
    

# print('Colaborative filtering recomendations:                      ', get_collaborative_filtering_recommendations_new(user_id, user_matrix_df, article_matrix_df))

# Combine content-based and collaborative filtering
def hybrid_recommendations(article_id, cosine_similarity_df): #,user_id, user_matrix_df, article_matrix_df 
    # Get content-based recommendations
    content_recs = get_content_based_recommendations(article_id, cosine_similarity_df)

    # # Get collaborative filtering recommendations
    # cf_recs = get_collaborative_filtering_recommendations_new(user_id, user_matrix_df, article_matrix_df)

    # Combine recommendations
    # combined_recs = list(content_recs) + list(cf_recs)

    combined_recs = content_recs
    
    return combined_recs#[:20]

# Example usage (We pass User_id and article_id that has in view in order to see if we will click it)
combined_recommendations =  hybrid_recommendations(article_id,cosine_similarity_df ) #, user_id,  user_matrix_df, article_matrix_df

print('Combined recommendations:                                   ', combined_recommendations)

Content based recomendations:                                   article_id  cosine_similarity
0     9776041           0.894936
1     9776862           0.888198
2     9777007           0.883037
3     9777319           0.876222
4     9733695           0.875143
5     9778007           0.866136
6     9739344           0.860317
7     9770051           0.856664
8     9739043           0.854321
9     9769387           0.853347
Combined recommendations:                                       article_id  cosine_similarity
0     9776041           0.894936
1     9776862           0.888198
2     9777007           0.883037
3     9777319           0.876222
4     9733695           0.875143
5     9778007           0.866136
6     9739344           0.860317
7     9770051           0.856664
8     9739043           0.854321
9     9769387           0.853347


In [331]:
counter, precisions, recalls, ndcgs, K = 0, [], [], [], 20



recommendations_full_lst = []

for _, row in tqdm(behaviors_df.iterrows()):

    user_id = row['user_id']
    article_ids_inview = row['article_ids_inview_setted_lst']

    recommendations_df = pd.DataFrame(columns=['article_id', 'cosine_similarity'])

    for article_id_inview in article_ids_inview:

        combined_recommendations = hybrid_recommendations(article_id_inview, cosine_similarity_df)

        recommendations_df = pd.concat([recommendations_df, combined_recommendations], ignore_index=True) #, user_id,  user_matrix_df, article_matrix_df 
        
        
    # recommendations_full_lst.append(recommendations)

    recommendations_df = recommendations_df.sort_values(by='article_id', ascending=False).head(50)

    recommendations_df = recommendations_df.sort_values(by='cosine_similarity', ascending=False).head(40)

    # recommendations_df.sort_values(by=['article_id', 'cosine_similarity'], ascending=[False, False], inplace=True)

    recommendations = recommendations_df.drop_duplicates(subset='article_id', keep='first')['article_id'].to_list()

    y_true = set(row['article_ids_clicked'])
    y_pred =set(recommendations[:K])

    # Calculate true positives (TP) and false positives (FP)
    true_positives = len(y_true & y_pred)
    false_positives = len(y_pred - y_true)

    # Calculate false negatives (FN)
    false_negatives = len(y_true - y_pred)

    # Calculate precision
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    # Calculate recall
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

    recommendations_full_lst.append(y_pred)
    
    # precision = len(set(y_true).intersection(set(y_pred))) / len(set(y_pred)) if len(set(y_pred)) > 0 else 0
    # recall = len(set(y_true).intersection(set(y_pred))) / len(y_true) if len(y_true) > 0 else 0
    
    precisions.append(precision)
    recalls.append(recall)
    #ndcgs.append(ndcg_score([y_true], [y_pred], k=K))

    if counter == 0:
        print(recommendations_df.head(2))
        counter += 1


    print(f"User: {user_id}, Lenght of Recoomendation: {len(recommendations)}, Percision: {precision}, Recall: {recall}  ")

print({
        'precision@K': sum(precisions) / len(precisions)
        ,'recall@K': sum(recalls) / len(recalls)
        #,'ndcg@K': sum(ndcgs) / len(ndcgs)
    })

# # Save the recommendation list
# recommendations_full_lst_file_path = f'./files/pickle/recommendations_behaviors_{val_size}_factorized_{factorized_size}_fillnan_{fillnan_value}_{mult_way}.pkl'

# with open(recommendations_full_lst_file_path, 'wb') as f:
0#   pickle.dump(recommendations_full_lst, f)

/var/folders/ld/c07vs56s18bc0h2krlvl0myr0000gn/T/ipykernel_3848/2679138338.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _, row in tqdm(behaviors_df.iterrows()):


0it [00:00, ?it/s]

    article_id  cosine_similarity
300    9778369           0.985444
80     9777079           0.984688
User: 11313, Lenght of Recoomendation: 34, Percision: 0.0, Recall: 0.0  
User: 13538, Lenght of Recoomendation: 38, Percision: 0.0, Recall: 0.0  
User: 15430, Lenght of Recoomendation: 38, Percision: 0.0, Recall: 0.0  
User: 19181, Lenght of Recoomendation: 28, Percision: 0.0, Recall: 0.0  
User: 19568, Lenght of Recoomendation: 39, Percision: 0.0, Recall: 0.0  
User: 19892, Lenght of Recoomendation: 40, Percision: 0.0, Recall: 0.0  
User: 19993, Lenght of Recoomendation: 40, Percision: 0.0, Recall: 0.0  
User: 21271, Lenght of Recoomendation: 38, Percision: 0.0, Recall: 0.0  
User: 21774, Lenght of Recoomendation: 36, Percision: 0.0, Recall: 0.0  
User: 22779, Lenght of Recoomendation: 30, Percision: 0.05, Recall: 0.058823529411764705  
User: 22895, Lenght of Recoomendation: 23, Percision: 0.05, Recall: 0.023809523809523808  
User: 24184, Lenght of Recoomendation: 23, Percision: 0.0, 

0

In [319]:
behaviors_df['article_ids_inview_setted_lst'][0]

[9776897,
 9779713,
 9775489,
 9779205,
 9777036,
 9778444,
 9778701,
 9774733,
 9776916,
 9776917,
 9779860,
 9776406,
 9778718,
 9776287,
 9775518,
 9766434,
 9776808,
 9773868,
 9776046,
 9776566,
 9775419,
 9779263,
 9779648,
 9779777,
 9778623,
 9778627,
 9778628,
 9775551,
 9767490,
 9776071,
 9776968,
 9775562,
 9776715,
 9775567,
 9759955,
 9777492,
 9780181,
 9775573,
 9776855,
 9758424,
 9775577,
 9776862,
 9776864,
 9780193,
 9778657,
 9779427,
 9778661,
 9776870,
 9775461,
 9777005,
 9778669,
 9775985,
 9776882,
 9777397,
 9776246,
 9779577,
 9778682,
 9775484]

In [314]:
behaviors_df['article_ids_clicked'][0]

[9776897, 9779860, 9778627, 9776855, 9776715, 9778701, 9775567, 9775489]

In [142]:
y_true_ex = {1,2,3}
y_pred_ex = {2,3,4}

In [145]:
len(y_true_ex.intersection(y_pred_ex)) /  len(y_pred_ex)

0.6666666666666666

In [141]:
precisions

[0.0,
 0.0,
 0.0,
 0.00539291217257319,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0037831021437578815,
 0.004827586206896552,
 0.010046044370029301,
 0.0024875621890547263,
 0.0068415051311288486,
 0.0,
 0.00546448087431694,
 0.027777777777777776,
 0.0030425963488843813,
 0.005571030640668524,
 0.0015128593040847202,
 0.001394700139470014,
 0.005521472392638037,
 0.0033222591362126247,
 0.004347826086956522,
 0.0037220843672456576,
 0.0,
 0.0020408163265306124,
 0.004336513443191674,
 0.0,
 0.0,
 0.0,
 0.0,
 0.005942733657482442,
 0.0045871559633027525,
 0.0,
 0.0010362694300518134,
 0.0,
 0.0,
 0.0019646365422396855,
 0.0,
 0.009547738693467337,
 0.0,
 0.0,
 0.0035545023696682463,
 0.0036319612590799033,
 0.0,
 0.004329004329004329,
 0.0,
 0.0,
 0.0,
 0.0012836970474967907,
 0.0,
 0.004492362982929021,
 0.0,
 0.003278688524590164,
 0.0021344717182497333,
 0.015895443306252206,
 0.0,
 0.0,
 0.0,
 0.003816793893129771,
 0.012422360248447204,
 0.0026857654431512983,
 0.0033519553072625698,
 0.0,


In [22]:
len(set(recommendations_full_lst[0]))

508

In [6]:
precisions[-5:]

[0.002362204724409449, 0.002577319587628866, 0.0006289308176100629, 0.0, 0.0]

In [26]:
# Load the recommendation list
factorized_size = 'small'
fillnan_value = 'mean_column'
recommendations_full_lst_file_path = f'./files/pickle/recommendations_behaviors_{val_size}_factorized_{factorized_size}_fillnan_{fillnan_value}_new.pkl'
with open(recommendations_full_lst_file_path, 'rb') as f:
   recommendations_full_lst = pickle.load(f)

In [21]:
behaviors_df['recommendations_demo'] = recommendations_full_lst

In [22]:
behaviors_df.head()

,user_id,article_ids_inview,article_ids_clicked,article_ids_inview_len,article_ids_clicked_len,recommendations_demo
0,19181,"[9783019, 9778732, 9783122, 9783024, 9783159, ...","[9783019, 9770145, 9782519, 9782899, 9788947, ...",368,27,"[9527400, 9503165, 9723467, 9710429, 9683183, ..."
1,21271,"[9784097, 7594265, 9782407, 9785267, 9784852, ...","[9785113, 9339920, 9786222, 9782407, 9784097, ...",98,7,"[9795545, 9591053, 9594387, 9278688, 9278910, ..."
2,21774,"[9780815, 9786111, 9777339, 9052390, 9785500, ...",[9785835],20,1,"[9787176, 9529717, 9163240, 9078202, 9577939, ..."
3,22779,"[9787465, 9787261, 9787332, 9787469, 9786495, ...","[9787261, 9784852, 9780702, 9785111, 9782806, ...",87,9,"[9788705, 9783720, 9570811, 9750749, 9787931, ..."
4,22895,"[9782652, 9783024, 9782202, 9783122, 9782996, ...","[9782996, 9790574, 9790475, 9781013, 9780514, ...",356,45,"[9520180, 9374607, 8924107, 9015511, 9607094, ..."


In [26]:
behaviors_df['article_ids_clicked'][7]

[9781998,
 9783852,
 9781983,
 9782282,
 9782407,
 9788898,
 9788106,
 9780702,
 9789065,
 9788411,
 9790682,
 9781624,
 9781598,
 9786222,
 9786279,
 9785404,
 9789832,
 9790825,
 9788393,
 9784064,
 9786567,
 9786245,
 9785016,
 9789702,
 9789537,
 9789692,
 9787659,
 9789471,
 9788576,
 9789418,
 9788323,
 9786549,
 9780374,
 9783314,
 9790521,
 9788151,
 9785829,
 9785899,
 9782635,
 9782808,
 9780815]

In [23]:
# Save the interaction matrix to pickle
size = 'demo'
behaviors_val_file_path = f'./files/pickle/behaviors_val_{size}.pkl'

behaviors_df.to_pickle(behaviors_val_file_path)

In [ ]:


# Function to evaluate recommendations
def evaluate_recommendations(validation_df, hybrid_recommendations, K=10):
    precisions, recalls, ndcgs = [], [], []
    
    for _, row in validation_df.iterrows():
        user_id = row['user_id']
        article_id = row['article_id']
        recommendations = hybrid_recommendations(user_id, true_article_id, user_matrix_df, article_matrix_df, cosine_similarity_df)
        
        y_true = [true_article_id]
        y_pred = recommendations[:K]
        
        precision = len(set(y_true).intersection(set(y_pred))) / len(y_pred) if len(y_pred) > 0 else 0
        recall = len(set(y_true).intersection(set(y_pred))) / len(y_true) if len(y_true) > 0 else 0
        
        precisions.append(precision)
        recalls.append(recall)
        ndcgs.append(ndcg_score([y_true], [y_pred], k=K))
    
    return {
        'precision@K': sum(precisions) / len(precisions),
        'recall@K': sum(recalls) / len(recalls),
        'ndcg@K': sum(ndcgs) / len(ndcgs)
    }

# Example evaluation
metrics = evaluate_recommendations(validation_df, hybrid_recommendations)
print(metrics)


In [37]:
a = [1,2,3]
b = [2,3,4]

len(set(a).intersection(set(b)))

2